In [5]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default ="plotly_white"

data = pd.read_csv("/content/rfm_data.csv")
print(data.head())

from datetime import datetime

# Converted PurchaseDate to Datetime
data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'])

#Calculated Recency
data['Recency']=(datetime.now().date()-data['PurchaseDate'].dt.date).dt.days

#Calculated Frequency
frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID':'Frequency'},inplace=True)
data = data.merge(frequency_data, on='CustomerID', how='left')

#Calculated Monetary Value
monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount':'MonetaryValue'},inplace=True)
data = data.merge(monetary_data, on='CustomerID', how='left')

#Defined Scoring criteria for each RFM Value
recency_scores =[5,4,3,2,1] #Higher score for lower recency
frequency_scores =[1,2,3,4,5] #Higher score for Higher frequency
monetary_scores =[1,2,3,4,5] #Higher score for higher monetary value

#Calculated RFM scores
data['RecencyScore'] =pd.cut(data['Recency'], bins=5, labels=recency_scores)
data['FrequencyScore'] =pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] =pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)

#Converted RFM scores to numeric type
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)

#Calculated RFM score by combining the individual scores
data['RFM_Score'] = data['RecencyScore']+ data['FrequencyScore']+ data['MonetaryScore']

#Created RFM segments based on the RFM score
segment_labels =['Low-Value','Mid_value','High_value']
data['value Segment'] = pd.qcut(data['RFM_Score'], q=3, labels =segment_labels)

#RFM Segment Distribution
segment_counts = data['value Segment'].value_counts().reset_index()
segment_counts.columns = ['value Segment','Count']
pastel_colors= px.colors.qualitative.Pastel

#Creating bar chart
fig_segment_dist =px.bar(segment_counts,x='value Segment',y='Count',color='value Segment',color_discrete_sequence=pastel_colors,title='RFM Value Segment Distribution')

#Updating the layout
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',yaxis_title='Count',showlegend=False)

fig_segment_dist.show()


#Creating new column for RFM customer Segments
data['RFM Customer Segments']=''

# Assign RFM Segments based on RFM score
data.loc[data['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
data.loc[(data['RFM_Score'] >= 6) & (data['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
data.loc[(data['RFM_Score'] >= 5) & (data['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
data.loc[(data['RFM_Score'] >= 4) & (data['RFM_Score'] < 5), 'RFM Customer Segments'] = 'Cant Lose'
data.loc[(data['RFM_Score'] >= 3) & (data['RFM_Score'] < 4), 'RFM Customer Segments'] = 'Lost'


#Creating visualization for RFM segments
segment_product_counts = data.groupby(['value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')
segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)
fig_treemap_segment_product = px.treemap(segment_product_counts,
                                        path=['value Segment', 'RFM Customer Segments'],
                                        values='Count',
                                        color='value Segment',
                                        color_discrete_sequence=px.colors.qualitative.Pastel,
                                        title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()

#Filtering the data to include only the customers in the Champions segment
champions_segment = data[data['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)


#Understanding the correlation between the recency,frequency and monetary scores within the champion segment

correlation_matrix = champions_segment[['RecencyScore','FrequencyScore','MonetaryScore']].corr()

#Visualizing the correlation matrix with the heatmap
fig_heatmap = go.Figure(data=go.Heatmap(
                  x=correlation_matrix.columns,
                  y=correlation_matrix.columns,
                  z=correlation_matrix.values,
                  colorscale="RdBu",
                  colorbar=dict(title='Correlation')))
fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_heatmap.show()

#Looking at the numbers of customers in all the segments
import plotly.colors
pastel_colors = plotly.colors.qualitative.Pastel
segment_counts = data['RFM Customer Segments'].value_counts()

#Creating a bar chart to compare segment counts
comparison_fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

champions_color = 'rgb(158, 202, 225)'
comparison_fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)


comparison_fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

comparison_fig.show()


# Calculating the average Recency, Frequency, and Monetary scores for each segment
segment_scores = data.groupby('RFM Customer Segments')['RecencyScore', 'FrequencyScore', 'MonetaryScore'].mean().reset_index()

# Create a grouped bar chart to compare segment scores
fig = go.Figure()

# Adding bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['RecencyScore'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'
))

# Adding bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['FrequencyScore'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))

# Adding bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['MonetaryScore'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))

# Updating the layout
fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()


#Creating the Dashboard
!pip install dash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.io as pio
import plotly.colors as pc

# Initializing the Dash app
app = dash.Dash(__name__)

# Definiing the app layout using Bootstrap components
app.layout = html.Div([
    html.H1("RFM Analysis Dashboard", className="text-center mb-4"),
    html.Div("Analyze customer segments based on RFM scores.", className="text-center mb-4"),

    # Dropdown for selecting the chart
    dcc.Dropdown(
        id='chart-type-dropdown',
        options=[
            {'label': 'RFM Value Segment Distribution', 'value': 'segment_distribution'},
            {'label': 'Distribution of RFM Values within Customer Segment', 'value': 'RFM_distribution'},
            {'label': 'Correlation Matrix of RFM Values within Champions Segment', 'value': 'correlation_matrix'},
            {'label': 'Comparison of RFM Segments', 'value': 'segment_comparison'},
            {'label': 'Comparison of RFM Segments based on Scores', 'value': 'segment_scores'},
        ],
        value='segment_distribution',  # Default selection
        className="mb-4",
    ),

    # Graph container
    dcc.Graph(id='rfm-chart', className="mb-4"),
])

# Defining callback to update the selected chart
@app.callback(
    Output('rfm-chart', 'figure'),
    [Input('chart-type-dropdown', 'value')]
)
def update_chart(selected_chart_type):
    if selected_chart_type == 'segment_distribution':
        return fig_segment_dist
    elif selected_chart_type == 'RFM_distribution':
        return fig_treemap_segment_product
    elif selected_chart_type == 'correlation_matrix':
        return fig_heatmap
    elif selected_chart_type == 'segment_comparison':
        return comparison_fig
    elif selected_chart_type == 'segment_scores':
        return fig

    # Return a default chart if no valid selection
    return fig_segment_dist

if __name__ == '__main__':
    app.run_server(port=8052)

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location  
0     Tokyo  
1    London  
2  New York  
3    London  
4     Paris  


<ipython-input-5-863d23a726bf>:139: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 51.1 MB/s eta 0:00:00


<IPython.core.display.Javascript object>